4. Написать python скрипт для агрегации данных из поднятой БД в результирующий .csv файл содержащий следующий датасет:
- день
- количество созданных новых аккаунтов
- количество сообщений написанных анонимами в процентах от всех сообщений
- количество сообщений всего
- процентное изменение количества тем на форуме относительно предыдущего дня (например относительно вчерашнего дня сегодня прирост количества тем 2% - вчера было всего 100 тем, сегодня создали 2 новых)

4.1 На вход скрипту должен подаваться период рассчета агрегации в любом выбранном формате


In [63]:
import mysql.connector
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta

In [8]:
# Подключение к MySQL
config = {
    'host': "localhost",
    'user': "root",
    'password': "root",
    'database': "log_db",
    'port': 3305
}
conn = mysql.connector.connect(**config)
df = pd.read_sql("SELECT * FROM logs", conn)
conn.close()
df

C:\Users\obbir\AppData\Local\Temp\ipykernel_15428\2022320230.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM logs", conn)


,id,user_id,action,time,response,anonym
0,1,0,first_visit,2025-02-01 01:12:00,Успешно!,1
1,2,0,first_visit,2025-02-01 10:54:00,Успешно!,1
2,3,0,first_visit,2025-02-01 22:35:00,Успешно!,1
3,4,0,first_visit,2025-02-01 12:41:00,Успешно!,1
4,5,0,first_visit,2025-02-01 03:33:00,Успешно!,1
...,...,...,...,...,...,...
1734,1735,0,create_message,2025-02-28 13:45:00,Успешно!,1
1735,1736,12,create_message,2025-02-28 10:32:00,Успешно!,0
1736,1737,0,create_message,2025-02-28 09:53:00,Успешно!,1
1737,1738,0,create_message,2025-02-28 02:05:00,Успешно!,1


In [86]:
period_start = input("Введите начальную дату периода в формате YYYY-MM-DD") #2025-02-05
period_end = input("Введите конечную дату периода в формате YYYY-MM-DD") #2025-02-25

In [87]:
period_start = datetime.strptime(period_start, '%Y-%m-%d')
period_end = datetime.strptime(period_end, '%Y-%m-%d')

In [98]:
#Отбираем дни
df['time'] = pd.to_datetime(df['time']).dt.date
df = df[(df['time'] <= period_end.date()) & (df['time'] >= period_start.date())]

days = df['time'].unique().tolist()
days = [day.strftime('%Y-%m-%d') for day in days]
days

['2025-02-05',
 '2025-02-06',
 '2025-02-07',
 '2025-02-08',
 '2025-02-09',
 '2025-02-10',
 '2025-02-11',
 '2025-02-12',
 '2025-02-13',
 '2025-02-14',
 '2025-02-15',
 '2025-02-16',
 '2025-02-17',
 '2025-02-18',
 '2025-02-19',
 '2025-02-20',
 '2025-02-21',
 '2025-02-22',
 '2025-02-23',
 '2025-02-24',
 '2025-02-25']

In [102]:
#кол-во созданных аккаунтов
temp = df[df['action'] == 'registration'].groupby('time').size().reset_index(name='count')
new_accs = temp['count'].tolist()

#кол-во сообщений всего
temp = df[df['action'] == 'create_message'].groupby('time').size().reset_index(name='count')
all_messages_count = temp['count'].tolist()

#кол-во сообщений, написанных анонимами (в %)
temp = df[(df['action'] == 'create_message') & (df['anonym'] == 1)].groupby('time').size().reset_index(name='count')
anonym_messages_count = temp['count'].tolist()

anonym_messages_percentage = (np.round((np.array(anonym_messages_count) / np.array(all_messages_count))*100, 2)).tolist()

#процентное изменение кол-ва тем на форуме
temp = df[df['action'] == 'create_theme'].groupby('time').size().reset_index(name='count')
create_theme_amount = temp['count'].tolist()

temp = np.round(pd.Series(create_theme_amount).pct_change() * 100, 2)
theme_change_percentage = temp.tolist()

In [104]:
output = pd.DataFrame.from_dict({
    "day": days,
    "registered": new_accs,
    "all_messages": all_messages_count,
    "anonym_messages_%": anonym_messages_count,
    "theme_change_%": theme_change_percentage
})
output.to_csv(header=True, sep = ';', index=False, path_or_buf='Task_1/data_files/output.csv', encoding='utf-8')